## AWS tutorial 'Capturing Table Activity with DynamoDB Streams'

When a record is inserted to a dynamoDB table, the lambda publisha message. An email is subscribed to the topic.

Setting up:
- create table
- create role, policy
- create topic, subscribe topic
- create lambda
- create lambda trig

<a href='#test_rec'>Test: adding records to DynamoDB</a>

https://docs.aws.amazon.com/amazondynamodb/latest/developerguide/Streams.Lambda.Tutorial.html?shortFooter=true#Streams.Lambda.Tutorial.CreateTable


In [5]:
aws dynamodb create-table --table-name DeviceInfoTable \
    --attribute-definitions AttributeName=DeviceId,AttributeType=S AttributeName=Timestamp,AttributeType=S \
    --key-schema AttributeName=DeviceId,KeyType=HASH  AttributeName=Timestamp,KeyType=RANGE \
    --provisioned-throughput ReadCapacityUnits=1,WriteCapacityUnits=1 \
    --stream-specification StreamEnabled=true,StreamViewType=NEW_AND_OLD_IMAGES

{
    "TableDescription": {
        "AttributeDefinitions": [
            {
                "AttributeName": "DeviceId",
                "AttributeType": "S"
            },
            {
                "AttributeName": "Timestamp",
                "AttributeType": "S"
            }
        ],
        "TableName": "DeviceInfoTable",
        "KeySchema": [
            {
                "AttributeName": "DeviceId",
                "KeyType": "HASH"
            },
            {
                "AttributeName": "Timestamp",
                "KeyType": "RANGE"
            }
        ],
        "TableStatus": "CREATING",
        "CreationDateTime": 1572371391.744,
        "ProvisionedThroughput": {
            "NumberOfDecreasesToday": 0,
            "ReadCapacityUnits": 1,
            "WriteCapacityUnits": 1
        },
        "TableSizeBytes": 0,
        "ItemCount": 0,
        "TableArn": "arn:aws:dynamodb:us-west-2:408575476948:table/DeviceInfoTable",
        "TableId": "8cdc42dc-a5d4-4e6a

In [6]:
# change wcu and rcu
# aws dynamodb update-table --table-name BarkTable --provisioned-throughput ReadCapacityUnits=1,WriteCapacityUnits=1

#describe
aws dynamodb describe-table --table-name DeviceInfoTable

{
    "Table": {
        "AttributeDefinitions": [
            {
                "AttributeName": "DeviceId",
                "AttributeType": "S"
            },
            {
                "AttributeName": "Timestamp",
                "AttributeType": "S"
            }
        ],
        "TableName": "DeviceInfoTable",
        "KeySchema": [
            {
                "AttributeName": "DeviceId",
                "KeyType": "HASH"
            },
            {
                "AttributeName": "Timestamp",
                "KeyType": "RANGE"
            }
        ],
        "TableStatus": "ACTIVE",
        "CreationDateTime": 1572371391.744,
        "ProvisionedThroughput": {
            "NumberOfDecreasesToday": 0,
            "ReadCapacityUnits": 1,
            "WriteCapacityUnits": 1
        },
        "TableSizeBytes": 0,
        "ItemCount": 0,
        "TableArn": "arn:aws:dynamodb:us-west-2:408575476948:table/DeviceInfoTable",
        "TableId": "8cdc42dc-a5d4-4e6a-84fc-84ba84f

In [7]:
# create trust-relation role
aws iam create-role --role-name DeviceLambdaRole --path "/service-role/" \
    --assume-role-policy-document file://trust-relationship.json

{
    "Role": {
        "Path": "/service-role/",
        "RoleName": "DeviceLambdaRole",
        "RoleId": "AROAV6IH7ATKH5IG4CBXD",
        "Arn": "arn:aws:iam::408575476948:role/service-role/DeviceLambdaRole",
        "CreateDate": "2019-10-29T18:00:06Z",
        "AssumeRolePolicyDocument": {
            "Version": "2012-10-17",
            "Statement": [
                {
                    "Effect": "Allow",
                    "Principal": {
                        "Service": "lambda.amazonaws.com"
                    },
                    "Action": "sts:AssumeRole"
                }
            ]
        }
    }
}


In [12]:
# create policy and attach it to the role
# aws iam put-role-policy --role-name DeviceLambdaRole --policy-name DeviceLambdaRolePolicy \
#    --policy-document file://role-policy.json

# describe
aws iam get-role-policy --role-name DeviceLambdaRole --policy-name DeviceLambdaRolePolicy

{
    "RoleName": "DeviceLambdaRole",
    "PolicyName": "DeviceLambdaRolePolicy",
    "PolicyDocument": {
        "Version": "2012-10-17",
        "Statement": [
            {
                "Effect": "Allow",
                "Action": "lambda:InvokeFunction",
                "Resource": "arn:aws:lambda:us-west-2:408575476948:function:publishDeviceStatus*"
            },
            {
                "Effect": "Allow",
                "Action": [
                    "logs:CreateLogGroup",
                    "logs:CreateLogStream",
                    "logs:PutLogEvents"
                ],
                "Resource": "arn:aws:logs:us-west-2:408575476948:*"
            },
            {
                "Effect": "Allow",
                "Action": [
                    "dynamodb:DescribeStream",
                    "dynamodb:GetRecords",
                    "dynamodb:GetShardIterator",
                    "dynamodb:ListStreams"
                ],
                "Resource": "arn:aws:dyna

In [13]:
# create a sns topic
aws sns create-topic --name deviceStatusTopic

{
    "TopicArn": "arn:aws:sns:us-west-2:408575476948:deviceStatusTopic"
}


In [14]:
# create topic subscription
aws sns subscribe --topic-arn arn:aws:sns:us-west-2:408575476948:deviceStatusTopic \
    --protocol email --notification-endpoint coderming@gmail.com

{
    "SubscriptionArn": "pending confirmation"
}


#### I create function 'publishDeviceStatus' at console

to use aws cli (not tested) : 
- edit publishDeviceStatus.py file
- then run:
<code>
$ zip publishDeviceStatus.zip publishDeviceStatus.py
$ aws lambda create-function \
    --region us-west-2 \
    --function-name publishDeviceStatus \
    --zip-file fileb://publishDeviceStatus.zip \
    --role arn:aws:iam::408575476948:role/service-role/DeviceLambdaRole \
    --handler publishDeviceStatus.handler \
    --timeout 5 \
    --runtime python3.7
</code>

In [15]:
# create dynamodb trig on the lambda
aws lambda create-event-source-mapping \
    --region us-west-2 \
    --function-name publishDeviceStatus \
    --event-source arn:aws:dynamodb:us-west-2:408575476948:table/DeviceInfoTable/stream/2019-10-29T17:49:51.744 \
    --batch-size 1 \
    --starting-position TRIM_HORIZON

{
    "UUID": "f3063ab1-9870-44ce-9130-7bb5058b04c7",
    "BatchSize": 1,
    "MaximumBatchingWindowInSeconds": 0,
    "EventSourceArn": "arn:aws:dynamodb:us-west-2:408575476948:table/DeviceInfoTable/stream/2019-10-29T17:49:51.744",
    "FunctionArn": "arn:aws:lambda:us-west-2:408575476948:function:publishDeviceStatus",
    "LastModified": 1572380863.114,
    "LastProcessingResult": "No records processed",
    "State": "Creating",
    "StateTransitionReason": "User action"
}


###  <a name='test_rec'>Test the flow by adding records to dynamodb </a>

In [18]:

aws dynamodb put-item --table-name DeviceInfoTable \
 --item DeviceId={S="DroneSimulator_D"},Timestamp={S="2019-10-29:13:10:36"},Status={S="OK"}
aws dynamodb put-item --table-name DeviceInfoTable \
 --item DeviceId={S="DroneSimulator_D"},Timestamp={S="2019-10-29:13:10:46"},Status={S="Bad"}
aws dynamodb put-item --table-name DeviceInfoTable \
 --item DeviceId={S="DroneSimulator_D"},Timestamp={S="2019-10-29:13:11:36"},Status={S="OK"}

In [ ]:
### connectIoT QMTT